In [7]:
import matplotlib.pyplot as plt

from matplotlib import animation
from IPython.display import HTML

from simulator import CartPoleSimulator

%matplotlib inline

simulation_time = 10 # seconds
time_step = 1/25 # freq 20hz
delay = 1 # seconds

freq = 0.2 # hz
amplitude = 0.20 # meter

env = CartPoleSimulator()
env.reset()

# prepare plots
figure, ax = plt.subplots(figsize=(12, 8));

# cart pose + action
ax = plt.subplot(211)
plt.title('Control')
ax.set_ylim(-3.0, 3.0)
ax.set_xlim(0, simulation_time)

moments = []

actions = []
action_plot, = plt.plot([],[], c='b', label='action')

cart_positions = []
cart_position_plot, = plt.plot([],[], c='g', label='position')

rewards = []
reward_plot, = plt.plot([],[], c='r', label='reward')
plt.grid(linestyle=':')
plt.legend()

plt.subplot(212)
plt.title('Camera')
plt.grid(linestyle=':')
canvas = plt.imshow(env.render(), zorder=1)


def init():
    return canvas, action_plot, cart_position_plot, reward_plot

def animate(t):
    _2PI = np.pi*2
    # get current state and action
    action = amplitude * np.sin(_2PI * freq * t)
    state = env.state()
        
    if state:
      # make a snapshot
      moments.append(t)
      actions.append(action / amplitude)
      cart_positions.append(state.cart_position / amplitude)
                
      next_state, reward, finish, info = env.step(action)
      rewards.append(reward)
    
      # make a normalized visualization
      canvas.set_data(env.render())
      action_plot.set_data(moments, actions)
      cart_position_plot.set_data(moments, cart_positions)
      reward_plot.set_data(moments, rewards)
    
    return canvas, action_plot, cart_position_plot, reward_plot

print('Please, wait a few seconds...')
result = animation.FuncAnimation(
    fig=figure,
    func=animate,
    init_func=init,
    frames=np.arange(0, simulation_time, time_step),
    interval=time_step*1000, # ms
    repeat_delay=delay*1000, # ms
    repeat=True
)

plt.close() # disable plot
HTML(result.to_jshtml())